# Distributed IO and cluster sizes

## Core Lesson

In a distributed file system, spreading IO to as many nodes as possible might speed up your computations.

## Set up a Slurm cluster

In [1]:
from dask.distributed import Client
from dask_jobqueue import SLURMCluster

/p/project/cecam/rath1/miniconda3_20190521/envs/dask_jobqueue_workshop/lib/python3.7/site-packages/docrep/__init__.py:341: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
  s = dedents('\n' + '\n'.join(lines[first:]))


In [2]:
import os

In [3]:
cluster = SLURMCluster(
    cores=24,
    processes=2,
    memory="100GB",
    shebang='#!/usr/bin/env bash',
    queue="batch",
    walltime="00:30:00",
    local_directory='/tmp',
    death_timeout="15s",
    interface="ib0",
    log_directory=f'{os.environ["SCRATCH_cecam"]}/{os.environ["USER"]}/dask_jobqueue_logs/',
    project="ecam",
    name="resilient_clusters")

In [4]:
client = Client(cluster)
client

Client Scheduler: tcp://10.80.32.41:39894 Dashboard: http://10.80.32.41:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


## The job scripts

In [5]:
print(cluster.job_script())

#!/usr/bin/env bash

#SBATCH -J resilient_clusters
#SBATCH -e /p/scratch/cecam/rath1/dask_jobqueue_logs//resilient_clusters-%J.err
#SBATCH -o /p/scratch/cecam/rath1/dask_jobqueue_logs//resilient_clusters-%J.out
#SBATCH -p batch
#SBATCH -A ecam
#SBATCH -n 1
#SBATCH --cpus-per-task=24
#SBATCH --mem=94G
#SBATCH -t 00:30:00
JOB_ID=${SLURM_JOB_ID%;*}



/p/project/cecam/rath1/miniconda3_20190521/envs/dask_jobqueue_workshop/bin/python -m distributed.cli.dask_worker tcp://10.80.32.41:39894 --nthreads 12 --nprocs 2 --memory-limit 50.00GB --name resilient_clusters--${JOB_ID}-- --death-timeout 15s --local-directory /tmp --interface ib0



## Scale the cluster to a single node

In [6]:
cluster.scale(2)  # will lead to one node

## Create some data and dump it to disk

In [7]:
from dask import array as da

In [8]:
random_numbers = da.random.uniform(0, 1, size=(1/8 * 500e9, 1), chunks=(1/8 * 200e6, 1))
random_numbers

dask.array<uniform, shape=(62500000000, 1), dtype=float64, chunksize=(25000000, 1)>

In [13]:
!rm -rf temporary_random_numbers.zarr/

In [14]:
%%time

random_numbers.to_zarr("temporary_random_numbers.zarr");

CPU times: user 31.7 s, sys: 2.19 s, total: 33.9 s
Wall time: 3min 26s


In [15]:
print(random_numbers.nbytes / 1e9, "GB")

500.0 GB


In [16]:
!du -sh temporary_random_numbers.zarr

406G	temporary_random_numbers.zarr


## Re-read and average

In [17]:
reread_random_numbers = da.from_zarr("temporary_random_numbers.zarr/")
reread_random_numbers

dask.array<zarr, shape=(62500000000, 1), dtype=float64, chunksize=(25000000, 1)>

In [18]:
print(reread_random_numbers.nbytes / 1e9, "GB")

500.0 GB


In [19]:
%%time

print(reread_random_numbers.mean().compute())

0.49999963422700916
CPU times: user 31.5 s, sys: 2.34 s, total: 33.9 s
Wall time: 1min 46s


## Scale the cluster to four nodes

In [20]:
cluster.scale(8)  # will lead to four nodes

## Re-read and average again

In [21]:
reread_random_numbers = da.from_zarr("temporary_random_numbers.zarr/")
reread_random_numbers

dask.array<zarr, shape=(62500000000, 1), dtype=float64, chunksize=(25000000, 1)>

In [22]:
print(reread_random_numbers.nbytes / 1e9, "GB")

500.0 GB


In [23]:
%%time

print(reread_random_numbers.mean().compute())

0.49999963422700916
CPU times: user 16.6 s, sys: 1.25 s, total: 17.8 s
Wall time: 23.5 s


## How far can we take this?

In [33]:
cluster.scale(40)  # will lead to 20 nodes

## Re-read and average again

In [34]:
reread_random_numbers = da.from_zarr("temporary_random_numbers.zarr/")
reread_random_numbers

dask.array<zarr, shape=(62500000000, 1), dtype=float64, chunksize=(25000000, 1)>

In [35]:
print(reread_random_numbers.nbytes / 1e9, "GB")

500.0 GB


In [36]:
%%time

print(reread_random_numbers.mean().compute())

0.49999963422700916
CPU times: user 8.24 s, sys: 579 ms, total: 8.82 s
Wall time: 9.11 s


## Summary

We have seen that IO bandwidth increases with the number of nodes we use for the computation.  This is obvious on one hand, but it tells us that from an analytics perspective where a lot of data lying around on disk is analysed, spreading out resources to as many nodes as possible may be the way to go.

In [ ]:
!rm -rf temporary_random_numbers.zarr/

## Complete listing of software used here

In [37]:
%pip list

Package            Version          
------------------ -----------------
asciitree          0.3.3            
aspy.yaml          1.2.0            
backcall           0.1.0            
bokeh              1.1.0            
certifi            2019.3.9         
cfgv               1.6.0            
cftime             1.0.3.4          
Click              7.0              
cloudpickle        1.0.0            
cycler             0.10.0           
cytoolz            0.9.0.1          
dask               1.2.0            
dask-jobqueue      0.4.1+32.g9c3371d
decorator          4.4.0            
distributed        1.27.1           
docrep             0.2.5            
fasteners          0.14.1           
heapdict           1.0.0            
identify           1.4.3            
importlib-metadata 0.13             
ipykernel          5.1.1            
ipython            7.5.0            
ipython-genutils   0.2.0            
jedi               0.13.3           
Jinja2             2.10.1           
j

In [38]:
%conda list --explicit

# This file may be used to create an environment using:
# $ conda create --name <env> --file <this file>
# platform: linux-64
@EXPLICIT
https://conda.anaconda.org/conda-forge/linux-64/git-lfs-2.7.2-0.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/ca-certificates-2019.3.9-hecc5488_0.tar.bz2
https://repo.anaconda.com/pkgs/main/linux-64/libgcc-ng-8.2.0-hdf63c60_1.tar.bz2
https://repo.anaconda.com/pkgs/main/linux-64/libgfortran-ng-7.3.0-hdf63c60_0.tar.bz2
https://repo.anaconda.com/pkgs/main/linux-64/libstdcxx-ng-8.2.0-hdf63c60_1.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/bzip2-1.0.6-h14c3975_1002.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/expat-2.2.5-hf484d3e_1002.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/icu-58.2-hf484d3e_1000.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/jpeg-9c-h14c3975_1001.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/libffi-3.2.1-he1b5a44_1006.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/